## Cálculo Recursivo Precisão Infinita (MRPI)

### Librarias

In [1]:
import sympy as sp
import numpy as np
from mpmath import *
x = sp.Symbol('x')

### Funções auxiliares

In [2]:

def matrizDosCoeficentes(objeto, n, prec):
    x = sp.Symbol('x')
    mp.dps = prec
    an = sp.Matrix( np.zeros( (n + 1) ) )
    fx = sp.Matrix(np.zeros( (1, n + 1) ) )
    if type( objeto) == list or type( objeto) == tuple:
        if len(objeto) >= n + 1:
            coluna = 0
            iteracoes = 0
            while(coluna <= n and iteracoes < 10**9):       
                an[coluna] = mpf(objeto[ coluna]) 
                fx[coluna] = x**coluna
                coluna += 1
                iteracoes += 1
                Sn = fx * an
            return( Sn, an )
        else:
            print('Número de coeficientes insuficente.')
    else:
        x = sp.Symbol('x')
        coluna = 0
        iteracoes = 0
        while(coluna <= n and iteracoes < 10**9):       
            derivada = sp.diff( objeto, x, coluna )
            derivadaEmZero = derivada.subs( x, mpf(0) ) 
            an[coluna] = mpf( sp.N(derivadaEmZero, prec) / sp.factorial(coluna)) 
            fx[coluna] = x**coluna
            coluna += 1
            iteracoes += 1
            Sn = fx * an
        return( Sn, an )
    
    return

### Cálculo do Padé  \[ p / q \]

In [5]:


def padeRecursivoPrecisaoFinita( objeto, n, iterações, percurso, prec):
    mp.dps = prec
    #Validar número de iterações
    max_iteraçoes = 2*n -1
    if(iterações>max_iteraçoes):
        return('Número de iterações inválido.')
    
    # Numeradores e denominadores dos aproximantes de Padé 
    numerador = sp.Matrix(np.zeros((2+iterações)))
    denominador = sp.Matrix(np.zeros((2+iterações)))
    
    # defnir o nome das variáveis recursivas como funções 
    for i in range(0,2+iterações):
            Ni = sp.Function('N'+str(i))
            numerador[i] = Ni
            Di = sp.Function('D'+str(i)) 
            denominador[i] = Di

    # Input: f_(n) , 1 ; f_(n-1) , 1
    
    p = matrizDosCoeficentes( objeto, n, prec)
    numerador[0] = p[0]
    denominador[0] = 1
    q = matrizDosCoeficentes(objeto, n-1, prec)
    numerador[1] = q[0]
    denominador[1] = 1
   
    # Guardar os aproximantes construídos
    pade = sp.Matrix(np.zeros((2+iterações)))
    
    #Padé [n/0]
    pade[0] = numerador[0]/denominador[0]
 
    #Padé [(n-1)/0]
    pade[1] = numerador[1]/denominador[1]
    
    
    # Iniciar o algoritmo de Baker
    i = 2
    j = 2
    
    while(i < j + iterações):
        
        cc0 = sp.Poly(numerador[i-2], x)
        c0 = cc0.coeffs()[0]
      
        cc1 = sp.Poly(numerador[i-1], x)
        c1 = cc1.coeffs()[0]
        
        # Expressões recursivas
        numerador[i] = sp.expand(sp.simplify((c1*numerador[i-2]-x*c0*numerador[i-1])/c1))
        denominador[i] = sp.expand(sp.simplify((c1*denominador[i-2]-x*c0*denominador[i-1])/c1))
        pade[i] = numerador[i]/denominador[i]
        
            
        i += 1
        if (i < j + iterações):
            cc0 = sp.Poly(numerador[i-2], x)
            c0 = cc0.coeffs()[0]
            cc1 = sp.Poly(numerador[i-1], x)
            c1 = cc1.coeffs()[0]
            
            # Expressões recursivas
            numerador[i] = sp.expand(sp.simplify((c1*numerador[i-2]-c0*numerador[i-1])/(c1-c0)))
            denominador[i] = sp.expand(sp.simplify((c1*denominador[i-2]-c0*denominador[i-1])/(c1-c0)))
            pade[i] = numerador[i]/denominador[i]
            
            
        i += 1
    # Caminho na tabela de Padé    
    if ( percurso == 1 ):
        return ( pade )
    
    # Último aproximante de Padé construído
    elif (percurso == 0):
        return(pade[-1])
    #erro
    else:
        return('O parâmetro percurso toma apenas os valores: 0 ou 1.')